In [1]:
# import lib
from IPython.display import Latex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torchsummary
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Function
import  visdom
import scipy.sparse as sp
import gurobipy as gp
import itertools
import pdb
import datetime
from random import choice
from gurobipy import GRB
import pickle

# Prepare data

In [2]:
def load_data(data_dir='1_layer_weights/256/'):
    dict_param = {}
    dict_param['w0'] = np.sign(np.load(data_dir+"w0.npy")).T
    dict_param['size_hidden'], dict_param['size_input'] = np.shape(dict_param['w0'])
    dict_param['w1'] = np.sign(np.load(data_dir+"w1.npy")).T    
    dict_param['size_output'],_ = np.shape(dict_param['w1'])

    # MNIST dataset
    test_dataset = torchvision.datasets.MNIST(root='data',
                                              train=False,
                                              transform=transforms.ToTensor())
    return dict_param, test_dataset

In [3]:
def set_param(dict_param, test_dataset, restriction=False,  flip_rate=0.5):
    x, label = choice(test_dataset)
    x = x.reshape(28*28).numpy()*2-1
    x = np.sign(x)
    x[x==0] = 1 
    confidence_vec = dict_param['w1']@np.sign(dict_param['w0']@x)
    label_predicted = np.argmax(confidence_vec)
    if restriction is True:
        flips = 2*np.random.binomial(n=1, p=1-flip_rate, size=dict_param['size_input'])-1
        x_turb = x * flips
        target = np.argmax(dict_param['w1']@np.sign(dict_param['w0']@x_turb))
        dict_param['set_const'] = np.nonzero(flips==1)[0]
        dict_param['set_var'] = np.nonzero(flips != 1)[0]
        print('the number of flips is {}'.format(len(dict_param['set_var'])))
    else:
        dict_param['set_var'] = range(dict_param['size_input'])
        dict_param['set_const'] = range(0)
        select_range = np.nonzero(confidence_vec<np.max(confidence_vec))[0] # avoid equal confidence level
        #target = (label_predicted + np.random.randint(9) + 1) % 10
        if len(select_range) == 0:
            raise Exception('Weird! All confidence levels are same!')
        target = select_range[np.random.randint(len(select_range))]
    print(label, label_predicted, target)
    dict_param['target'] = target
    dict_param['label_original'] = label
    dict_param['label_predicted'] = label_predicted
    dict_param['x'] = x

# Build MIP model

In [16]:
class MinistExtendedFormulation(object):
    def __init__(self, dict_param, is_conv=False, is_integral=False, is_branchPriority=False,
                 obj_type=GRB.MINIMIZE, pixel_change_tol=784, model_attrs={}):
        self.model = gp.Model("robustness_BNN")
        self.param = dict_param
        self.is_conv = is_conv
        self.is_integral= is_integral
        self.is_branchPriority = is_branchPriority
        self.obj_type = obj_type
        self.pixel_change_tol = pixel_change_tol
        self.model_attrs = model_attrs
        
        self.vars = {}
        self.vars_incumbent = dict(Status=False)
        self.confidence_level = 0.
        self.result_only = True # save the result only
    
    def setup(self):
        """set up the corresponding model
        """
        self.model.setParam("TimeLimit", 1800)
        self.model.setParam("PreCrush", 1)
#         if self.is_integral is False:
#             #choose barrier method for continuous models
#             self.model.setParam("Method", 2)
#             #Disable crossover to accelarate the algorithm
#             self.model.setParam("Crossover", 0)
            
        for key,value in self.model_attrs.items():
            self.model.setParam(key, value)
            
        model = self.model
        w0 = self.param['w0']
        w1 = self.param['w1']
        x = self.param['x']
        
        model._param = self.param
        model._num_usercut = 0
        
        set_var= self.param['set_var']
        size_hidden = self.param['size_hidden']
        size_output = self.param['size_output']
        
        label_predicted = dict_param['label_predicted']
        target = self.param['target']
        
        type_var = GRB.BINARY if self.is_integral else GRB.CONTINUOUS
        
        x0 = model.addVars(set_var, vtype=type_var, ub=1., name='x0')
        if self.is_branchPriority is True:
            # branch on lower-lever variables first
            for i in x0:
                x0[i].branchPriority = 1
                
        x1 = model.addVars(range(size_hidden), vtype=type_var, ub=1., name='x1')
        
        y0 = model.addVars(set_var, lb=-1., vtype=GRB.CONTINUOUS, name='y0')
        y1 = model.addVars(range(size_hidden), lb=-1., vtype=GRB.CONTINUOUS, name='y1')
        
        self.vars['y0'] = y0
        self.vars['y1'] = y1
        model._vars = self.vars
        #self.vars['z0'] = z0
        
        model.addConstrs(y0[j] == 2*x0[j] - 1. for j in set_var)
        model.addConstrs(y1[j] == 2*x1[j] - 1. for j in range(size_hidden))
        
        if self.is_conv is True:
            self.add_constrs_conv()
        else:
            self.add_constrs_natural()
            
        if self.obj_type == GRB.MINIMIZE:
            model.addConstrs(gp.quicksum(w1[j, k]*y1[k] for k in range(size_hidden)) + self.confidence_level <= 
                          gp.quicksum(w1[target, k]*y1[k] for k in range(size_hidden)) for j in range(size_output) if j != target)
            model.setObjective(len(set_var)/2-gp.quicksum(x[j]*y0[j]/2 for j in set_var), GRB.MINIMIZE)
        elif self.obj_type == GRB.MAXIMIZE:
            model.addConstr(len(set_var)-gp.quicksum(x[j]*y0[j] for j in set_var) <= 2*self.pixel_change_tol)
            model.setObjective(gp.quicksum(w1[target, k]*y1[k] for k in range(size_hidden)) - 
                               gp.quicksum(w1[label_predicted, k]*y1[k] for k in range(size_hidden)), GRB.MAXIMIZE)
    
    def set_confidence_level(self, confidence_level):
        self.confidence_level = confidence_level
        self.model = gp.Model("robustness_BNN")
        self.vars_incumbent = dict(Status=False)
        
    def add_constrs_natural(self):
        model = self.model
        set_var = self.param['set_var']
        set_const = self.param['set_const']
        size_hidden = self.param['size_hidden'] 
        size_input = self.param['size_input'] 
        w = self.param['w0']
        name_vars = ['y0','y1']
        y0,y1 = (self.vars[name] for name in name_vars)
        z_const = self.param['w0'][:, set_const]
        y_const = z_const @ self.param['x'][set_const]
        model.addConstrs(gp.quicksum(w[i,j]*y0[j] for j in set_var) + y_const[i] + size_input >= size_input * y1[i]
                         for i in range(size_hidden))
        model.addConstrs(gp.quicksum(w[i,j]*y0[j] for j in set_var) + y_const[i] - size_input + 1. <= size_input * y1[i]
                         for i in range(size_hidden))
        
    def add_constrs_conv(self):
        model = self.model
        row = self.param['size_hidden'] 
        w = self.param['w0']
        name_vars = ['y0','y1']
        y0,y1= (self.vars[name] for name in name_vars)
        col = self.param['size_input']
             
        set_var = self.param['set_var']
        set_const = self.param['set_const']
        z_const = self.param['w0'][:, set_const]
        y_const = z_const @ self.param['x'][set_const]   
        z_aux = model.addVars(range(row), set_var, lb=-1.0, vtype=GRB.CONTINUOUS)
        
        model.addConstrs(-(1.+y1[i])<=2*z_aux[i,j] for i in range(row) for j in set_var)
        model.addConstrs(2*z_aux[i,j]<=(1.+y1[i]) for i in range(row) for j in set_var)
        model.addConstrs(-(1-y1[i])<=2*(w[i,j]*y0[j]-z_aux[i,j]) for i in range(row) for j in set_var)
        model.addConstrs(2*(w[i,j]*y0[j]-z_aux[i,j])<=(1-y1[i]) for i in range(row) for j in set_var)
        model.addConstrs(2*z_aux.sum(i, '*') + y_const[i] * (1+y1[i])>=0 for i in range(row))
        model.addConstrs(2*(gp.quicksum(w[i,j]*y0[j] for j in set_var)-z_aux.sum(i,'*'))
                         + (y_const[i]+2) * (1-y1[i])<= 0 for i in range(row))
        
#         z1 = model.addVars(range(row), range(col), lb=-1.0, ub=1., vtype=GRB.CONTINUOUS)
#         z2 = model.addVars(range(row), range(col), lb=-1.0, ub=1., vtype=GRB.CONTINUOUS)
#         model.addConstrs(z1[i,j]<=w[i,j]*y0[j] for i in range(row) for j in range(col))
#         model.addConstrs(z1[i,j]<=y1[i] for i in range(row) for j in range(col))
#         model.addConstrs(z2[i,j]>=w[i,j]*y0[j] for i in range(row) for j in range(col))
#         model.addConstrs(z2[i,j]>=y1[i] for i in range(row) for j in range(col))
#         model.addConstrs(col*(y1[i]-1)<=2*gp.quicksum(z1[i,j] for j in range(col)) for i in range(row))
#         model.addConstrs((col+2)*y1[i]+col-2>=2*gp.quicksum(z2[i,j] for j in range(col)) for i in range(row))
        
    def __repr__(self):
        status = self.vars_incumbent['Status']
        s = ''
        if status is False:
            s = s + 'The optimization problem has not been solved. Please try to solve the problem first.'
        elif status != GRB.OPTIMAL:
            s = s + 'The solver is not able to solve the problem to optimality.'
        else:
            s = s + ('The solver is able to solve the problem to optimality within {}.\n'.format(self.vars_incumbent['time']))
            s = s + ('The original input is:\n{}\n'.format(self.param['x']))
            y0,y1 = (self.vars_incumbent[name] for name in ['y0', 'y1'])
            s= s + ('The targeted input is:\n{}\n'.format(y0))
            s= s + ('The hidden layers are:\n{}\n'.format(y1))
        return s
    
    def __call__(self, callback=None):
        if self.vars_incumbent['Status'] is False:
            self.setup()          
            
        if callback is None:
            self.model.optimize()
        elif self.is_integral is False:
            self.model.setParam("OutputFlag", 0)
            self.model._tol = self.model.Params.FeasibilityTol
            #self.model._tol = 1e-5
            self.model._iter_tol = 10000
            self.model._iter = 0
            #self.model._lazy_status = False
            obj_previous = -1e5
            obj_current = 1e5
            while np.abs(obj_previous-obj_current)>self.model._tol*1e-2:
            #while (self.model._lazy_status is False) and (np.abs(obj_previous-obj_current)<=self.model._tol):
                self.model.optimize()
                obj_previous = obj_current
                obj_current = self.model.objVal
                callback(self.model)
        else:
            self.model._tol = 0
            self.model.optimize(callback)
            #self.model.optimize(conv_cb)
        self.vars_incumbent['Status'] = self.model.getAttr('Status')
        if self.vars_incumbent['Status'] == GRB.OPTIMAL:
            print('The objective is {}'.format(self.model.objVal))
            self.set_optimal_sols()
        
    def set_optimal_sols(self, is_print=True):
        model = self.model
        size_hidden = self.param['size_hidden']
        set_var = self.param['set_var'] 
        names_vars = ['y0','y1']
        y0,y1 = (self.vars[name] for name in names_vars)
        
        self.vars_incumbent['y0'] = np.array(model.getAttr("X", y0).values())
        self.vars_incumbent['y1'] = np.array(model.getAttr("X", y1).values())
        self.vars_incumbent['objVal'] = model.objVal
        self.vars_incumbent['time'] = model.getAttr('Runtime')
    
    def save(self, filename, result_only=True):
        model = self.model
        x = self.param['x']
        target = self.param['target']
        label_original = self.param['label_original']
        label_predicted = self.param['label_predicted']
        names_incumbent = ['y0','y1','objVal','time', 'Status']
        dict_instance = {'confidence_level':self.confidence_level, 'label_original':label_original,
                     'label_predicted':label_predicted, 'target':target, 'time':self.model.getAttr('Runtime')}
        try:
            y0,y1,objVal,time,status = (self.vars_incumbent[name] for name in names_incumbent)
            dict_instance.update({'x':x, 'y0':y0, 'y1':y1, 'time':time, 'Status':status, 'objVal':objVal})
        except:
            dict_instance['Status'] = -1
            
        if self.is_integral is True:
            try:
                dict_instance['gap'] = self.model.getAttr('MIPGap')
                dict_instance['bound'] = self.model.getAttr('ObjBound')
                dict_instance['objVal'] = self.model.getAttr('objVal')
                dict_instance['time'] = self.model.getAttr('Runtime')
                dict_instance['Status'] = self.model.getAttr('Status')
            except:
                print('MIP failed to save!')
                dict_instance = {'Status':-1}

        self.result_only = result_only
        if result_only is False:
            # model
            model.write(filename+'.mps')
            # MIP start
            model.write(filename+'.mst')
            # save solutinos
            model.write(filename+'.json')
            
            # save parameters
        with open(filename, 'wb') as f:
                pickle.dump(dict_instance, f)
    
    def load(self, filename):
        with open(filename, 'rb') as f:
            dict_instance = pickle.load(f)
        self.param['x'] = dict_instance['x']
        self.param['target'] = dict_instance['target']
        if self.result_only is False:
            self.model = gp.read(filename+'.mps')
            size_input = self.param['size_input']
            size_hidden = self.param['size_hidden'] 
            names_incumbent = ['y0','y1','objVal','time', 'Status']
            for name in names_incumbent:
                self.vars_incumbent[name] = dict_instance[name]
            self.model.read(filename+'.mst')
            self.vars['y0'] = dict(zip(range(size_input), ((x for x in self.model.getVars() if x.VarName.find('y0') != -1))))
            self.vars['y1'] = dict(zip(range(size_hidden), ((x for x in self.model.getVars() if x.VarName.find('y1') != -1))))
            self.model.update()

In [5]:
def conv_cb(model, where):
    if where == GRB.Callback.MIPNODE:
        status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
        #if (status == GRB.OPTIMAL):
        #if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)<= 100):
        # only adding cutting planes at the root node
        if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)==0):
            #pdb.set_trace()        
            m = model._param['size_hidden'] # number of constraints            
            n = model._param['size_input'] # number of variables
            weights = model._param['w0'] # weights matrix
            name_vars = ['y0','y1']
            y0,y1 = (model._vars[name] for name in name_vars)
            
            #pdb.set_trace()
            y0_incumbent = np.array(model.cbGetNodeRel(y0).select())
            y1_incumbent = np.array(model.cbGetNodeRel(y1).select())
            for i in range(m):
                w = weights[i,:]
                t = y1_incumbent[i]
                if np.sum(np.minimum(w*y0_incumbent, t)) < n/2*(t-1) - model._tol:
                    indices = np.where(w*y0_incumbent<t)[0]
                    # (|J|-n/2)t-n/2 <= \sum_{j\in J} w_j*y_j
                    model.cbCut((len(indices)-n/2)*y1[i] - n/2 <= gp.quicksum(weights[i,j]*y0[j] for j in indices))
                    model._num_usercut = model._num_usercut + 1

                if np.sum(np.maximum(w*y0_incumbent, t)) > (n+2)*(t+1)/2-2+model._tol:
                    indices = np.where(w*y0_incumbent>t)[0]
                    # (|J|+1-n/2)t+n/2-1 >= \sum_{j\in J} w_j*y_j
                    #pdb.set_trace()
                    model.cbCut((len(indices)-n/2+1)*y1[i]+n/2-1 >= gp.quicksum(weights[i,j]*y0[j] for j in indices))
                    model._num_usercut = model._num_usercut + 1

In [6]:
def conv_lazy(model, where):
    if where == GRB.Callback.MIPNODE:
        print('MIPNODE')
#         status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
#         if (status == GRB.OPTIMAL):
        #if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)<= 100):
        # only adding cutting planes at the root node
        #if (status == GRB.OPTIMAL) and (model.cbGet(GRB.Callback.MIPNODE_NODCNT)==0):
            #pdb.set_trace()        
        m = model._param['size_hidden'] # number of constraints            
        n = model._param['size_input'] # number of variables
        weights = model._param['w0'] # weights matrix
        name_vars = ['y0','y1']
        y0,y1 = (model._vars[name] for name in name_vars)
        var_auxiliary = model._vars['auxiliary']

        #pdb.set_trace()
        y0_incumbent = np.array(model.cbGetNodeRel(y0).select())
        y1_incumbent = np.array(model.cbGetNodeRel(y1).select())
        var_auxiliary_incumbent = np.array(model.cbGetNodeRel(var_auxiliary))
        for i in range(m):
            w = weights[i,:]
            t = y1_incumbent[i]
            if np.sum(np.minimum(w*y0_incumbent, t)) < var_auxiliary_incumbent + n/2*(t-1) - model._tol:
                indices = np.where(w*y0_incumbent<t)[0]
                # (|J|-n/2)t-n/2 <= \sum_{j\in J} w_j*y_j
                model.cbLazy((len(indices)-n/2)*y1[i] - n/2 + var_auxiliary <= gp.quicksum(weights[i,j]*y0[j] for j in indices))
                model._num_usercut = model._num_usercut + 1

            if np.sum(np.maximum(w*y0_incumbent, t)) + var_auxiliary_incumbent > (n+2)*(t+1)/2-2+model._tol:
                indices = np.where(w*y0_incumbent>t)[0]
                # (|J|+1-n/2)t+n/2-1 >= \sum_{j\in J} w_j*y_j
                #pdb.set_trace()
                model.cbLazy((len(indices)-n/2+1)*y1[i]+n/2-1 >= var_auxiliary + gp.quicksum(weights[i,j]*y0[j] for j in indices))
                model._num_usercut = model._num_usercut + 1

In [7]:
def add_cut(model):
    m = model._param['size_hidden'] # number of constraints            
    n = model._param['size_input'] # number of variables
    weights = model._param['w0'] # weights matrix
    name_vars = ['y0','y1']
    y0,y1 = (model._vars[name] for name in name_vars)

    #pdb.set_trace()
    y0_incumbent = np.array(model.getAttr("X", y0).values())
    y1_incumbent = np.array(model.getAttr("X", y1).values())
    model._iter = model._iter + 1
    #model._lazy_status = True
    for i in range(m):
        w = weights[i,:]
        t = y1_incumbent[i]
        if np.sum(np.minimum(w*y0_incumbent, t)) < n/2*(t-1) - model._tol:
            indices = np.where(w*y0_incumbent<t)[0]
            # (|J|-n/2)t-n/2 <= \sum_{j\in J} w_j*y_j
            model.addConstr((len(indices)-n/2)*y1[i] - n/2 <= gp.quicksum(weights[i,j]*y0[j] for j in indices))
            model._num_usercut = model._num_usercut + 1
            #model._lazy_status = False

        if np.sum(np.maximum(w*y0_incumbent, t)) > (n+2)*(t+1)/2-2+model._tol:
            indices = np.where(w*y0_incumbent>t)[0]
            # (|J|+1-n/2)t+n/2-1 >= \sum_{j\in J} w_j*y_j
            #pdb.set_trace()
            model.addConstr((len(indices)-n/2+1)*y1[i]+n/2-1 >= gp.quicksum(weights[i,j]*y0[j] for j in indices))
            model._num_usercut = model._num_usercut + 1
            #model._lazy_status = False

# Experiments with varying changing tolerance

In [17]:
obj_conv = {}
obj_natural = {}
obj_conv_int = {}
obj_natural_int = {}

# specify your own directory to place the experiment results
root_dir = 'experiment_results/robustness_integral_size_hidden1_max_with_varying_pixel_tol/'
for size_hidden in [ 512,]:
    data_dir = '1_layer_weights/' + str(size_hidden) + '/'
    dict_param, test_dataset = load_data(data_dir=data_dir)
    for idx_instance in range(5):
        set_param(dict_param, test_dataset)
        pixel_change_range = [2,3,4] # allowed to change about 5% of pixels
        print('Hidden size:{}, instance index:{}'.format(size_hidden, idx_instance+1)+'*'*40+'{}'.format(datetime.datetime.now()))
        for pixel_change_tol in pixel_change_range:
            print('Hidden size:{}, instance index:{}, tolerance:{}'.format(size_hidden, idx_instance+1, pixel_change_range)
                  +'*'*40+'{}'.format(datetime.datetime.now()))
            instance_natural = MinistExtendedFormulation(dict_param, is_integral=False,
                                                         obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n NATURAL RELAXATION STARTS TO SOLVE'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_natural()
            try:
                instance_natural.save(root_dir+'nat{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_natural[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_natural.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_natural[str(size_hidden), str(idx_instance)] = -1           


            instance_conv = MinistExtendedFormulation(dict_param, is_integral=False,
                                                      obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n STRONG RELAXATION STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_conv(add_cut)
            try:
                instance_conv.save(root_dir+'conv{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))
                obj_conv[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_conv.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_conv[str(size_hidden), str(idx_instance)] = -1

            instance_natural_int = MinistExtendedFormulation(dict_param, is_integral=True,
                                                             obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n NATURAL MIP STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_natural_int()
            try:
                instance_natural_int.save(root_dir+'nat_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_natural_int[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_natural_int.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_natural_int[str(size_hidden), str(idx_instance)] = -1

            instance_conv_int = MinistExtendedFormulation(dict_param, is_integral=True,
                                                          obj_type=GRB.MAXIMIZE, pixel_change_tol=pixel_change_tol)
            print('\n STRONG MIP STARTS TO SOLVE!'+'-'*20
                  +'# hidden neurons'+str(size_hidden)+','+'#instance'+str(idx_instance+1)+',tol'+str(pixel_change_tol)+'\n')
            instance_conv_int(conv_cb)
            try:
                instance_conv_int.save(root_dir+'conv_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol)))                              
                obj_conv_int[str(size_hidden), str(idx_instance), str(pixel_change_tol)] = instance_conv_int.vars_incumbent['objVal']
            except KeyError:
                print('Oops! The model is not optimal!')
                obj_conv_int[str(size_hidden), str(idx_instance)] = -1

1 1 0
Hidden size:512, instance index:1****************************************2021-05-14 14:44:34.775521
Hidden size:512, instance index:1, tolerance:[2, 3, 4]****************************************2021-05-14 14:44:34.776537

 NATURAL RELAXATION STARTS TO SOLVE--------------------# hidden neurons512,#instance1,tol2

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2321 rows, 2592 columns and 807216 nonzeros
Model fingerprint: 0x43f7680b
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 1808 rows and 1296 columns
Presolve time: 0.34s
Presolved: 513 rows, 1808 columns


Optimal solution found (tolerance 1.00e-04)
Best objective -8.800000000000e+01, best bound -8.800000000000e+01, gap 0.0000%
The objective is -88.0

 STRONG MIP STARTS TO SOLVE!--------------------# hidden neurons512,#instance1,tol2

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2321 rows, 2592 columns and 807216 nonzeros
Model fingerprint: 0xb8dae8b5
Variable types: 1296 continuous, 1296 integer (1296 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 1337 rows and 1318 columns (presolve time = 5s) ...
Presolve removed 1337 rows and 1315 columns
Presolve time: 6.40s
Presolved: 984 rows, 1277 columns, 759911 nonzeros
Variable types: 0 continuous, 1277 i

     0     0  -82.75171    0   56  -88.00000  -82.75171  5.96%     -   56s
     0     0  -82.75433    0   55  -88.00000  -82.75433  5.96%     -   56s
     0     0  -82.75433    0   57  -88.00000  -82.75433  5.96%     -   57s
     0     0  -82.82351    0   73  -88.00000  -82.82351  5.88%     -   57s
     0     0  -82.83469    0   71  -88.00000  -82.83469  5.87%     -   57s
     0     0  -82.84548    0   70  -88.00000  -82.84548  5.86%     -   57s
     0     0  -82.85653    0   68  -88.00000  -82.85653  5.84%     -   58s
     0     0  -82.85781    0   70  -88.00000  -82.85781  5.84%     -   58s
     0     0  -83.03896    0   72  -88.00000  -83.03896  5.64%     -   58s
     0     0  -83.43372    0   67  -88.00000  -83.43372  5.19%     -   58s
     0     0  -83.52110    0   76  -88.00000  -83.52110  5.09%     -   59s
     0     0  -83.53738    0   84  -88.00000  -83.53738  5.07%     -   59s
     0     0  -83.53945    0   84  -88.00000  -83.53945  5.07%     -   59s
     0     0  -83.72705  

     0     0  -46.35532    0  107  -68.00000  -46.35532  31.8%     -   37s
     0     0  -46.40817    0  103  -68.00000  -46.40817  31.8%     -   37s
     0     0  -46.48471    0  114  -68.00000  -46.48471  31.6%     -   39s
     0     0  -46.49655    0  115  -68.00000  -46.49655  31.6%     -   39s
     0     0  -46.55436    0  116  -68.00000  -46.55436  31.5%     -   41s
     0     0  -46.59533    0  121  -68.00000  -46.59533  31.5%     -   41s
     0     0  -46.67497    0  119  -68.00000  -46.67497  31.4%     -   43s
     0     0  -46.67497    0  119  -68.00000  -46.67497  31.4%     -   43s
     0     0  -46.68124    0   15  -68.00000  -46.68124  31.4%     -   46s
     0     0  -46.68124    0   69  -68.00000  -46.68124  31.4%     -   48s
     0     0  -46.68124    0   68  -68.00000  -46.68124  31.4%     -   49s
     0     0  -46.68124    0   63  -68.00000  -46.68124  31.4%     -   49s
     0     0  -46.86129    0   75  -68.00000  -46.86129  31.1%     -   50s
     0     0  -47.04459  

     0     0  -44.09173    0   93  -68.00000  -44.09173  35.2%     -   70s
     0     0  -44.09173    0  102  -68.00000  -44.09173  35.2%     -   72s
     0     0  -44.09428    0  108  -68.00000  -44.09428  35.2%     -   73s
     0     0  -44.09850    0  115  -68.00000  -44.09850  35.1%     -   74s
     0     0  -44.10189    0  116  -68.00000  -44.10189  35.1%     -   76s
     0     0  -44.12566    0  104  -68.00000  -44.12566  35.1%     -   78s
     0     0  -44.13232    0  106  -68.00000  -44.13232  35.1%     -   79s
     0     0  -44.13234    0  111  -68.00000  -44.13234  35.1%     -   81s
     0     0  -44.13247    0  112  -68.00000  -44.13247  35.1%     -   82s
     0     0  -44.13254    0  115  -68.00000  -44.13254  35.1%     -   84s
     0     0  -44.14782    0  108  -68.00000  -44.14782  35.1%     -   85s
     0     0  -44.15272    0  100  -68.00000  -44.15272  35.1%     -   87s
     0     0  -44.15272    0   65  -68.00000  -44.15272  35.1%     -   90s
     0     0  -44.15272  

...

Thread count was 16 (of 16 available processors)

Solution count 5: 2 -2 -14 ... -38

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
The objective is 2.0

 STRONG MIP STARTS TO SOLVE!--------------------# hidden neurons512,#instance3,tol4

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2321 rows, 2592 columns and 807216 nonzeros
Model fingerprint: 0x38d1160c
Variable types: 1296 continuous, 1296 integer (1296 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 1337 rows and 1321 columns (presolve time = 5s) ...
Presolve removed 1337 rows and 1315 columns
Presolve time: 6.49s
Presol

H    0     0                     -38.0000000   52.85860   239%     -    7s
     0     0  -23.11088    0    5  -38.00000  -23.11088  39.2%     -    8s
H    0     0                     -34.0000000  -23.11088  32.0%     -    8s

Cutting planes:
  Gomory: 2
  MIR: 5

Explored 1 nodes (360 simplex iterations) in 8.57 seconds
Thread count was 16 (of 16 available processors)

Solution count 4: -34 -38 -42 -46 
No other solutions better than -34

Optimal solution found (tolerance 1.00e-04)
Best objective -3.400000000000e+01, best bound -3.400000000000e+01, gap 0.0000%

User-callback calls 481, time in user-callback 0.62 sec
The objective is -34.0
Hidden size:512, instance index:4, tolerance:[2, 3, 4]****************************************2021-05-14 15:35:18.756591

 NATURAL RELAXATION STARTS TO SOLVE--------------------# hidden neurons512,#instance4,tol3

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   P

   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
The objective is 29.455322007760962

 NATURAL MIP STARTS TO SOLVE!--------------------# hidden neurons512,#instance4,tol4

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2321 rows, 2592 columns and 807216 nonzeros
Model fingerprint: 0xc893d123
Variable types: 1296 continuous, 1296 integer (1296 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 1337 rows and 1321 columns (presolve time = 5s) ...
Presolve removed 1337 rows and 1315 columns
Presolve time: 6.54s
Presolved: 984 rows, 1277 columns, 759911 nonzeros
Variable types: 0 co

     0     0   11.11890    0    9   -4.00000   11.11890   378%     -    8s
     0     0    4.00000    0   28   -4.00000    4.00000   200%     -    8s
H    0     0                      -0.0000000    4.00000      -     -    9s

Cutting planes:
  Gomory: 5
  Cover: 3
  MIR: 134
  StrongCG: 6
  Zero half: 24
  RLT: 1

Explored 1 nodes (496 simplex iterations) in 9.33 seconds
Thread count was 16 (of 16 available processors)

Solution count 4: -0 -4 -8 -36 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%
The objective is -0.0

 STRONG MIP STARTS TO SOLVE!--------------------# hidden neurons512,#instance5,tol2

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2321 rows, 2592 columns an

H    0     0                       4.0000000   18.16855   354%     -    9s
*    0     0               0      12.0000000   12.00000  0.00%     -    9s

Cutting planes:
  Gomory: 7
  MIR: 5
  Zero half: 39

Explored 1 nodes (499 simplex iterations) in 9.24 seconds
Thread count was 16 (of 16 available processors)

Solution count 4: 12 4 -4 -36 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.200000000000e+01, best bound 1.200000000000e+01, gap 0.0000%

User-callback calls 808, time in user-callback 0.65 sec
The objective is 12.0
Hidden size:512, instance index:5, tolerance:[2, 3, 4]****************************************2021-05-14 15:51:38.092664

 NATURAL RELAXATION STARTS TO SOLVE--------------------# hidden neurons512,#instance5,tol4

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a 

In [2]:
def format_decimal(x):    
    if x==np.inf:
        s = '-'
    else:
        s = '{:.2f}'.format(x)
    return s
def format_int(x):
    if x==np.inf:
        s = '-'
    else:
        s = '{:d}'.format(int(x))
    return s
experiment_frame = pd.DataFrame({'dim_hidden':[], 'pixel_change_tol':[], 'label':[],
                                 'nat_relax':[], 'nat_obj':[], 'nat_bound':[], 'nat_gap':[], 'nat_time':[],
                                 'conv_relax':[], 'conv_obj':[], 'conv_bound':[], 'conv_gap':[], 'conv_time':[], 'rimp':[]})
root_dir = 'experiment_results/robustness_integral_size_hidden1_max_with_varying_pixel_tol/'
n=0
for size_hidden in [32,64,128,256]:
    dim_hidden = size_hidden
    pixel_change_range = [1,2,3,4,5]
    for pixel_change_tol in pixel_change_range:
        for idx_instance in range(5):
            filename = root_dir + 'nat{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            label = (d['label_original'], d['label_predicted'], d['target'])
            nat_relax = d['objVal']

            filename = root_dir + 'conv{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
                
            try:
                conv_relax = d['objVal']
            except:
                conv_relax = np.nan
#             if d['Status'] != -1:
#                 conv_relax = d['objVal']
#             else:
#                 conv_relax = np.nan

            filename = root_dir + 'nat_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            nat_obj = d['objVal']
            nat_bound = d['bound']
            nat_gap = d['gap']*100
            nat_time = d['time']

            filename = root_dir + 'conv_int{}_{}_{}'.format(int(size_hidden), int(idx_instance), int(pixel_change_tol))
            with open(filename, 'rb') as f:
                d = pickle.load(f)
            conv_obj = d['objVal']
            conv_bound = d['bound']
            conv_gap = d['gap']*100
            conv_time = d['time']
            rimp = 100*(nat_relax-conv_relax)/(nat_relax-nat_obj + (np.abs(nat_relax-nat_obj)<=1e-4))

            list_frame = ['','', label,  nat_relax,  nat_obj, nat_bound, nat_gap, nat_time,
                                       conv_relax, conv_obj, conv_bound, conv_gap, conv_time, rimp]
            if idx_instance == 0:
                list_frame[0] = dim_hidden
                list_frame[1] = pixel_change_tol

            experiment_frame.loc[n] = list_frame
            n= n + 1
        list_frame = ['', '', 'Ave'] + list(experiment_frame[-5:].mean())
        experiment_frame.loc[n] = list_frame
        n = n + 1
    
        
formatters = [None,  None, None,format_decimal, format_int, format_int, format_int, format_decimal, format_decimal,
              format_int, format_int, format_int, format_decimal, format_decimal]        
latex_str = experiment_frame.to_latex(index=False, formatters=formatters)
print(latex_str)

C:\Users\Shaoning\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


\begin{tabular}{lllrrrrrrrrrrr}
\toprule
dim\_hidden & pixel\_change\_tol &      label & nat\_relax & nat\_obj & nat\_bound & nat\_gap & nat\_time & conv\_relax & conv\_obj & conv\_bound & conv\_gap & conv\_time &  rimp \\
\midrule
        32 &                1 &  (8, 8, 9) &     15.85 &      -4 &        -4 &       0 &     0.07 &       0.36 &       -4 &         -4 &        0 &      0.08 & 78.03 \\
           &                  &  (7, 7, 8) &     23.03 &     -12 &       -12 &       0 &     0.08 &      -9.09 &      -12 &        -12 &        0 &      0.08 & 91.68 \\
           &                  &  (8, 8, 0) &     26.12 &      -8 &        -8 &       0 &     0.08 &      -3.72 &       -8 &         -8 &        0 &      0.08 & 87.45 \\
           &                  &  (7, 1, 4) &     35.52 &     -16 &       -16 &       0 &     0.07 &      -7.93 &      -16 &        -16 &        0 &      0.07 & 84.33 \\
           &                  &  (8, 8, 1) &     17.82 &     -16 &       -16 &       0 &    

In [23]:
experiment_frame.to_csv(root_dir+'results.csv', index=False)